# Training Manager

This notebook manages the training of the finalized neural networks that we want to use for the pipeline.

## Data loading

First we must load the data and prepare it for training.

In [1]:
import sys
sys.path.append('../')

from wvz_ml_framework.nn_training import data_management

In [5]:
data_paths = {
    'Signal': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_VVZ.arrow',
    'Other': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_others.arrow',
    'ttZ': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_ttZ.arrow',
    'tWZ': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_tWZ.arrow',
    'tZ': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_tZ.arrow',
    'WZ': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_WZ.arrow',
    'Zgamma': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_Zgamma.arrow',
    'Zjets': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_Zjets.arrow',
    'ZZ': '/home/grabanal/WVZ/gabriel_ML_data/20220301_ELReLMIs54_MUReLMIs31_btag77_ZZ.arrow'
}

training_features_path = 'training_features_2022_06_01.txt'
training_features_rescale_params_path = training_features_path[:-4]+'_rescale_params.json'

with open(training_features_path, 'r') as file:
    training_features = [line.strip() for line in file.readlines()]
    
rescale_features = [feat for feat in training_features if feat not in ['SR', 'sr_DF', 
                                                                       'sr_SF_in_Z', 
                                                                       'sr_SF_no_Z']]

data_management.generate_scale_params_file(data_paths, rescale_features, 
                                           training_features_rescale_params_path)

Notice that if we train over every SR at the same time, the SR is one-hot encoded into the flags `'sr_DF'`, `'sr_SF_in_Z'`, and `'sr_SF_no_Z'`.

In [8]:
x_train, y_train, w_train, x_test, y_test, w_test, x_val, y_val, w_val \
    = data_management.get_train_test_val_data(data_paths=data_paths, 
                                              train_feats=training_features,
                                              sr_to_train='full',
                                              val_prop=0.1,
                                              test_prop=0.1,
                                              rescale_filepath=training_features_rescale_params_path,
                                              rescale_feats=rescale_features
                                             )

Data loaded...
Data scaled...
Signal regions one-hot encoded...
Splits generated... Finished.


## Model training

Now we can train the model. For reproducibility, it is important to draw training features from `.txt` files that have been stored down.

In [9]:
from wvz_ml_framework.nn_training import nn_training

2022-06-01 20:30:56.018709: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-01 20:30:56.805550: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-01 20:30:56.806872: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-01 20:30:56.820810: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-06-01 20:30:56.820840: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-01 20:30:56.823851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-0

In [12]:
nn_training.make_and_train_model(
    training_data=(x_train, y_train, w_train),
    validation_data=(x_val, y_val, w_val),
    batch_size=512,
    num_nodes=64,
    dropout=0.1,
    learn_rate=1e-5,
    epochs=10000,
    patience=200,
    model_dir='models/',
    model_name='model_2022_06_01',
    generate_onnx=True
)

Epoch 1/10000
3338/3338 [==============================] - 27s 8ms/step - loss: 0.0012 - accuracy: 0.8262 - val_loss: 8.7380e-04 - val_accuracy: 0.8132
Epoch 2/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 9.7574e-04 - accuracy: 0.7723 - val_loss: 7.9650e-04 - val_accuracy: 0.7530
Epoch 3/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 8.9661e-04 - accuracy: 0.7484 - val_loss: 7.7920e-04 - val_accuracy: 0.7581
Epoch 4/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 8.7411e-04 - accuracy: 0.7565 - val_loss: 7.6475e-04 - val_accuracy: 0.7663
Epoch 5/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 8.6103e-04 - accuracy: 0.7649 - val_loss: 7.1444e-04 - val_accuracy: 0.7884
Epoch 6/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 8.4238e-04 - accuracy: 0.7722 - val_loss: 7.2225e-04 - val_accuracy: 0.7869
Epoch 7/10000
3338/3338 [==============================] - 13s 4ms/s

3338/3338 [==============================] - 13s 4ms/step - loss: 7.3975e-04 - accuracy: 0.7891 - val_loss: 6.8343e-04 - val_accuracy: 0.7846
Epoch 54/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 7.3641e-04 - accuracy: 0.7884 - val_loss: 6.8945e-04 - val_accuracy: 0.7820
Epoch 55/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 7.4225e-04 - accuracy: 0.7845 - val_loss: 6.7830e-04 - val_accuracy: 0.7859
Epoch 56/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 7.3877e-04 - accuracy: 0.7850 - val_loss: 6.6105e-04 - val_accuracy: 0.7893
Epoch 57/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 7.4024e-04 - accuracy: 0.7855 - val_loss: 6.9276e-04 - val_accuracy: 0.7802
Epoch 58/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 7.3487e-04 - accuracy: 0.7841 - val_loss: 6.5226e-04 - val_accuracy: 0.7920
Epoch 59/10000
3338/3338 [==============================] - 13s 4ms/step 

3338/3338 [==============================] - 13s 4ms/step - loss: 6.8175e-04 - accuracy: 0.7802 - val_loss: 6.0210e-04 - val_accuracy: 0.7892
Epoch 106/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.8047e-04 - accuracy: 0.7839 - val_loss: 6.0924e-04 - val_accuracy: 0.7849
Epoch 107/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.8077e-04 - accuracy: 0.7800 - val_loss: 6.2901e-04 - val_accuracy: 0.7800
Epoch 108/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.8051e-04 - accuracy: 0.7802 - val_loss: 6.0246e-04 - val_accuracy: 0.7868
Epoch 109/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.7924e-04 - accuracy: 0.7810 - val_loss: 5.8417e-04 - val_accuracy: 0.7930
Epoch 110/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.8111e-04 - accuracy: 0.7823 - val_loss: 6.1265e-04 - val_accuracy: 0.7826
Epoch 111/10000
3338/3338 [==============================] - 13s 4ms

Epoch 157/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.2659e-04 - accuracy: 0.7944 - val_loss: 5.6396e-04 - val_accuracy: 0.7930
Epoch 158/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.2806e-04 - accuracy: 0.7931 - val_loss: 5.6030e-04 - val_accuracy: 0.7949
Epoch 159/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.1728e-04 - accuracy: 0.7961 - val_loss: 5.5067e-04 - val_accuracy: 0.7994
Epoch 160/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.2626e-04 - accuracy: 0.7950 - val_loss: 5.2983e-04 - val_accuracy: 0.8034
Epoch 161/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.2131e-04 - accuracy: 0.7940 - val_loss: 5.2859e-04 - val_accuracy: 0.8037
Epoch 162/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 6.1566e-04 - accuracy: 0.7971 - val_loss: 5.2391e-04 - val_accuracy: 0.8051
Epoch 163/10000
3338/3338 [=========================

Epoch 209/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.7377e-04 - accuracy: 0.8125 - val_loss: 4.9136e-04 - val_accuracy: 0.8192
Epoch 210/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.7808e-04 - accuracy: 0.8103 - val_loss: 4.7531e-04 - val_accuracy: 0.8245
Epoch 211/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.8066e-04 - accuracy: 0.8108 - val_loss: 4.6455e-04 - val_accuracy: 0.8278
Epoch 212/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.7734e-04 - accuracy: 0.8104 - val_loss: 4.8566e-04 - val_accuracy: 0.8219
Epoch 213/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.7909e-04 - accuracy: 0.8133 - val_loss: 5.0442e-04 - val_accuracy: 0.8133
Epoch 214/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.7792e-04 - accuracy: 0.8099 - val_loss: 4.9909e-04 - val_accuracy: 0.8173
Epoch 215/10000
3338/3338 [=========================

...

Epoch 677/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0259e-04 - accuracy: 0.8343 - val_loss: 4.2296e-04 - val_accuracy: 0.8501
Epoch 678/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0060e-04 - accuracy: 0.8356 - val_loss: 4.4103e-04 - val_accuracy: 0.8426
Epoch 679/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0223e-04 - accuracy: 0.8349 - val_loss: 4.4269e-04 - val_accuracy: 0.8436
Epoch 680/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0389e-04 - accuracy: 0.8338 - val_loss: 4.5919e-04 - val_accuracy: 0.8382
Epoch 681/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0518e-04 - accuracy: 0.8361 - val_loss: 4.4236e-04 - val_accuracy: 0.8425
Epoch 682/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0236e-04 - accuracy: 0.8352 - val_loss: 4.3739e-04 - val_accuracy: 0.8449
Epoch 683/10000
3338/3338 [=========================

Epoch 729/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 4.9840e-04 - accuracy: 0.8343 - val_loss: 4.4958e-04 - val_accuracy: 0.8419
Epoch 730/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 4.9907e-04 - accuracy: 0.8366 - val_loss: 4.5640e-04 - val_accuracy: 0.8388
Epoch 731/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 4.9688e-04 - accuracy: 0.8352 - val_loss: 4.3435e-04 - val_accuracy: 0.8455
Epoch 732/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 4.9712e-04 - accuracy: 0.8341 - val_loss: 4.4085e-04 - val_accuracy: 0.8433
Epoch 733/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 5.0253e-04 - accuracy: 0.8356 - val_loss: 4.5373e-04 - val_accuracy: 0.8375
Epoch 734/10000
3338/3338 [==============================] - 13s 4ms/step - loss: 4.9914e-04 - accuracy: 0.8344 - val_loss: 4.2571e-04 - val_accuracy: 0.8474
Epoch 735/10000
3338/3338 [=========================

2022-06-01 23:24:55.810403: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model_2022_06_01/assets
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-06-01 23:24:56.214353: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-06-01 23:24:56.214425: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-06-01 23:24:56.214969: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-06-01 23:24:56.215006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-01 23:24:56.215021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-01 23:24:56.215031: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-06-01 23:24:56.215040: I ten